In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sqlalchemy as sqla

engine = sqla.create_engine(sqla.URL.create(
    drivername="postgresql+psycopg2",
    host="127.0.0.1",
    username="indexor",
    password="indexor",
    database="indexor",
))

con = engine.connect()

figsize = (15, 12)

def set_legend(title: str):
    plt.legend(title=title, bbox_to_anchor=(1, 1))

def set_x_axis_legend():
    plt.locator_params(axis='x', nbins=20)
    plt.xticks(rotation=45)

# Blocks

## Coinbase rewards

In [ ]:
def block_reward(block_height):
    halving_interval = 210_000
    initial_reward = 50

    halvings = block_height // halving_interval

    if halvings >= 64:
        return 0

    return initial_reward / (2 ** halvings)


def show_coinbase():
    coinbase = pd.read_sql_query(sqla.text("""
SELECT
    b.height AS height,
    SUM(o.value) / POW(10, 8) AS coinbase
FROM transactions t
    INNER JOIN blocks b ON t.block = b.id
    INNER JOIN inputs i ON t.id = i.tx
    INNER JOIN outputs o ON t.id = o.tx
WHERE tx_in IS NULL
GROUP BY (b.height)
ORDER BY height;
    """), con)

    coinbase["rewards"] = coinbase.apply(lambda row: block_reward(row["height"]), axis=1)
    coinbase["fees"] = coinbase["coinbase"] - coinbase["rewards"]

    coinbase.plot(x="height", y=["rewards", "fees"], kind="area", figsize=figsize, stacked=True)
    set_legend("Type")
    set_x_axis_legend()
    plt.xlabel("Height")
    plt.ylabel("Block rewards")
    plt.title("Coinbase reward per block")


show_coinbase()

## Block times

In [ ]:
def show_block_times():
    times = pd.read_sql_query(sqla.text("""
SELECT
    height,
    (LEAD(time) OVER (ORDER BY height) - time) / 60.0 as block_time
FROM blocks
ORDER BY height;
    """), con)

    times['difficulty_adjustment_period'] = (times['height'] // 2016).astype(str)

    plt.figure(figsize=figsize)
    sns.boxplot(x='difficulty_adjustment_period', y='block_time', data=times)
    plt.xlabel('Difficulty adjustment period')
    plt.ylabel('Block time')
    plt.title('Block time by difficulty adjustment period')


show_block_times()

## Block size

In [ ]:
def show_block_size():
    coinbase = pd.read_sql_query(sqla.text("""
SELECT
    height,
    size,
    weight
FROM blocks
ORDER BY height;
    """), con)

    coinbase.plot(x="height", y=["weight", "size"], kind="line", figsize=figsize, stacked=False)
    set_legend("Type")
    set_x_axis_legend()
    plt.xlabel("Height")
    plt.ylabel("Block size")
    plt.title("Block sizes")


show_block_size()

# UTXO set

## UTXO set growth

In [ ]:
def show_utxo_growth():
    utxo = pd.read_sql_query(sqla.text("""
WITH txs AS (
    -- TODO: make this a materialized view
    SELECT
        t.id,
        t.txid,
        t.block,
        MAX(i.vin) + 1 AS vins,
        MAX(o.vout) + 1 AS vouts
    FROM transactions t
        INNER JOIN inputs i ON t.id = i.tx
        INNER JOIN outputs o ON t.id = o.tx
    GROUP BY t.id, t.txid, t.block
)
SELECT
    b.height,
    SUM(t.vins) AS inputs,
    SUM(t.vouts) AS outputs
FROM blocks b
    INNER JOIN txs t ON b.id = t.block
GROUP BY b.height
ORDER BY b.height;
    """), con)

    utxo["delta"] = utxo["outputs"] - utxo["inputs"]
    utxo["delta_sum"] = utxo["delta"].cumsum()
    print(utxo)

    utxo.plot(x="height", y="delta_sum", kind="area", stacked=False, legend=False, figsize=figsize)
    set_x_axis_legend()
    plt.xlabel("Height")
    plt.ylabel("UTXO set delta")
    plt.title("UTXO set growth")


show_utxo_growth()

# Outputs

## Output Types

In [ ]:
def show_output_types():
    types = pd.read_sql_query(sqla.text("""
WITH txs AS (
    SELECT
        b.height AS block,
        ot.name as type
    FROM transactions t
        INNER JOIN blocks b ON b.id = t.block
        INNER JOIN outputs o ON t.id = o.tx
        INNER JOIN output_types ot ON o.type = ot.id
), txs_per_block AS (
    SELECT
        block,
        COUNT(*) AS count
    FROM txs
    GROUP BY (block)
), block_type_count AS (
    SELECT
        block,
        type,
        COUNT(*) AS count
    FROM txs
    GROUP BY (block, type)
)
SELECT
    bt.block AS block,
    type,
    bt.count AS count,
    (bt.count * 100) / tp.count::DOUBLE PRECISION AS percent
FROM block_type_count bt
    INNER JOIN txs_per_block tp ON bt.block = tp.block
ORDER BY block, percent DESC;
    """), con)

    pivot_df = types.pivot(index="block", columns="type", values="percent")
    pivot_df.plot(kind="area", stacked=True, figsize=figsize)
    set_x_axis_legend()
    set_legend("Transaction types")
    plt.xlabel("Block")
    plt.ylabel("Percent")
    plt.title("Percent of output types per block")


show_output_types()

## Taproot adoption

In [ ]:
def show_taproot_percentage():
    outputs = pd.read_sql_query(sqla.text("""
WITH taproot_output AS (
    SELECT id
    FROM output_types
    WHERE name = 'witness_v1_taproot'
)
SELECT
    b.height,
    COUNT(*) outputs,
    COUNT(*) FILTER (
        WHERE o.type = (SELECT id FROM taproot_output)
    ) taproot_outputs
FROM blocks b
    INNER JOIN transactions t ON b.id = t.block
    INNER JOIN outputs o ON t.id = o.tx
GROUP BY b.height
ORDER by b.height;
    """), con)

    outputs["percentage_taproot"] = (outputs["taproot_outputs"] * 100) / outputs["outputs"]

    outputs.plot(x="height", y="percentage_taproot", kind="area", legend=False, figsize=figsize)
    set_x_axis_legend()
    plt.xlabel("Height")
    plt.ylabel("% Taproot outputs")
    plt.title("% of Taproot outputs per block")


show_taproot_percentage()

# Fee market

## Self transfer % of total weight

In [ ]:
def show_self_transfer():
    self_transfer = pd.read_sql_query(sqla.text("""
WITH block_summary AS (
    SELECT
        b.height,
        t.id,
        SUM(t.weight) AS total_weight,
        CASE
            WHEN (SELECT COUNT(*) FROM inputs i WHERE t.id = i.tx) = 1
                     AND (SELECT COUNT(*) FROM outputs o WHERE t.id = o.tx) = 1
            THEN t.weight
            ELSE 0
        END AS self_transfer_weight
    FROM transactions t
        INNER JOIN blocks b ON b.id = t.block
    GROUP BY (b.height, t.id)
)
SELECT
    height,
    (SUM(self_transfer_weight) * 100) /
        SUM(total_weight)::DOUBLE PRECISION AS percent_self_transfer
FROM block_summary
GROUP BY height
ORDER BY (height);
    """), con)

    self_transfer.plot(
        x="height",
        y="percent_self_transfer",
        kind="area",
        legend=False,
        figsize=figsize
    )
    set_x_axis_legend()
    plt.xlabel("Height")
    plt.ylabel("Self transfer %")
    plt.title("Self transfer % of total weight")


show_self_transfer()